<a href="https://colab.research.google.com/github/shhuangmust/AI/blob/master/Youtube_%E5%AD%97%E5%B9%95%E7%94%A2%E7%94%9F_%E6%96%B0%E7%89%88.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#安裝套件
!pip install -U yt-dlp --quiet
!pip install -U openai-whisper --quiet
!sudo apt-get install ffmpeg -y > /dev/null

In [ ]:
# 載入 Whisper 模型（使用 base/medium/large 視需求，可用 base/medium 最快）
import whisper
model = whisper.load_model("medium")  # 建議使用 'medium' 兼顧準確率與速度


In [ ]:
#輸入 YouTube 連結
youtube_url = 'https://www.youtube.com/watch?v=jbM3DRpR--w'

In [ ]:
# 使用 yt-dlp 下載音訊為 mp4 格式
import os

audio_filename = 'audio.mp4'
subtitle_file = 'subtitles.srt'

# 檢查並刪除舊檔
for file in [audio_filename, subtitle_file]:
    if os.path.exists(file):
        os.remove(file)
        print(f"🗑️ 已刪除舊檔：{file}")

# -f ba：最佳音訊，-x：抽出音訊，--audio-format mp4：輸出 mp4 音訊檔, -o：指定輸出檔案
cmd = f"yt-dlp -f bestaudio[ext=mp4] -o {audio_filename} {youtube_url}"
return_code = os.system(cmd)
print(return_code)

In [ ]:
import yt_dlp
import os

ydl_opts = {}
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    info = ydl.extract_info(youtube_url, download=False)
    video_title = info.get("title", "downloaded_video").replace('/', '_').replace('\\', '_')  # 防止非法字元
    audio_filename = f"{video_title}.mp4".replace(" ", "")
    subtitle_filename = f"{video_title}.srt".replace(" ", "")
# -f ba：最佳音訊，-x：抽出音訊，--audio-format mp4：輸出 mp4 音訊檔, -o：指定輸出檔案
cmd = f"yt-dlp -f bestaudio[ext=mp4] -o {audio_filename} {youtube_url}"
return_code = os.system(cmd)
print(audio_filename)


In [ ]:
result = model.transcribe(audio_filename, language='zh')
print("字幕轉換完成")

In [ ]:
# 儲存字幕為 SRT 格式
from datetime import timedelta

def format_timestamp(seconds):
    return str(timedelta(seconds=int(seconds))).zfill(8).replace('.', ',')

with open(subtitle_filename, "w", encoding="utf-8") as srt_file:
    for i, segment in enumerate(result["segments"]):
        start = format_timestamp(segment["start"])
        end = format_timestamp(segment["end"])
        text = segment["text"].strip()
        srt_file.write(f"{i + 1}\n{start} --> {end}\n{text}\n\n")

print(f"✅ 字幕已建立：{subtitle_filename}")